In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import yaml
from datetime import datetime
import numpy as np
import tensorflow as tf
from steel_seg.utils import (
    dice_coeff_kaggle,
    rle_to_dense,
    dense_to_rle,
    visualize_segmentations,
    onehottify)
from steel_seg.dataset.severstal_steel_dataset import SeverstalSteelDataset
from steel_seg.model.unet import build_unet_model
from steel_seg.model.classification_wrapper import build_classification_model
from steel_seg.train import (
    class_weighted_binary_classification_crossentropy,
    binary_accuracy_by_class)
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Necessary for CUDA 10 or something?
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"
os.environ["TF_ENABLE_AUTO_MIXED_PRECISION"] = "1"
os.environ["TF_ENABLE_AUTO_MIXED_PRECISION_GRAPH_REWRITE"] = "1"
os.environ["TF_ENABLE_AUTO_MIXED_PRECISION_LOSS_SCALING"] = "1"

In [ ]:
with open('SETTINGS.yaml') as f:
    cfg = yaml.load(f)

In [ ]:
dataset = SeverstalSteelDataset.init_from_config('SETTINGS.yaml')

In [ ]:
train_data, train_batches = dataset.create_dataset(dataset_type='training', dense_segmentation=False)
val_data, val_batches = dataset.create_dataset(dataset_type='validation', dense_segmentation=False)

In [ ]:
seg_model = build_unet_model(
    img_height=cfg['IMG_HEIGHT'],
    img_width=cfg['IMG_WIDTH'],
    img_channels=1,
    num_classes=cfg['NUM_CLASSES'],
    num_layers=4,
    activation=tf.keras.activations.elu,
    kernel_initializer='he_normal',
    kernel_size=(3, 3),
    pool_size=(2, 4),
    num_features=[32, 64, 128, 256],
    drop_prob=0.5)
model_checkpoint_name = 'deep'

In [ ]:
seg_model.summary()

In [ ]:
!ls checkpoints

In [ ]:
date_str = '20190916-092052'

In [ ]:
checkpoint_name = f'{model_checkpoint_name}_{date_str}'
checkpoint_path = f'checkpoints/{checkpoint_name}/cp-{checkpoint_name}' + '-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
initial_epoch = 0
if latest_checkpoint is None:
    print('Error. No checkpoints found.')
else:
    print(f'Loading weights from {latest_checkpoint}')
    last_epoch = latest_checkpoint.split('-')[-1]
    last_epoch = last_epoch.split('.')[0]
    initial_epoch = int(last_epoch)
    seg_model.load_weights(latest_checkpoint)

In [ ]:
cls_model = build_classification_model(seg_model, 'conv2d_7', 4, 'conv2d_14')
model_checkpoint_name = 'classifier' # TODO: cleanup this hack

In [ ]:
cls_model.summary()

In [ ]:
# train_imgs = dataset.get_image_list('training')
# class_counts = np.zeros((4,))
# for img_name in train_imgs:
#     img, ann = dataset.get_example_from_img_name(img_name)
#     class_counts += np.amax(ann, axis=(0, 1))
# np.array([len(train_imgs)] * 4) / class_counts

In [ ]:
#cls_weights = [30.0, 40.0, 10.0, 20.0]
cls_weights = [14.15538847, 53.28301887,  2.43921399, 15.62378976]

In [ ]:
cls_model.compile(
    optimizer=tf.train.AdamOptimizer(0.0001),
    loss={'classification_output': class_weighted_binary_classification_crossentropy(cls_weights)},
    metrics={'classification_output': [
        binary_accuracy_by_class(0),
        binary_accuracy_by_class(1),
        binary_accuracy_by_class(2),
        binary_accuracy_by_class(3),
    ]}
)

In [ ]:
!ls classification_checkpoints

In [ ]:
#date_str = '20190925-232140' # First try, probably not great weights
date_str = datetime.now().strftime("%Y%m%d-%H%M%S")
date_str

In [ ]:
checkpoint_name = f'{model_checkpoint_name}_{date_str}'
checkpoint_path = f'classification_checkpoints/{checkpoint_name}/cp-{checkpoint_name}' + '-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
initial_epoch = 0
if latest_checkpoint is None:
    print('No checkpoints found. Starting from scratch.')
else:
    print(f'Loading weights from {latest_checkpoint}')
    last_epoch = latest_checkpoint.split('-')[-1]
    last_epoch = last_epoch.split('.')[0]
    initial_epoch = int(last_epoch)
    cls_model.load_weights(latest_checkpoint)

In [ ]:
# Create checkpoint callback
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    monitor='val_loss',
    save_best_only=True,
    mode='auto',
    save_weights_only=True,
    verbose=1)


logdir = f'logs/{checkpoint_name}-{initial_epoch}'
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir=logdir),
    checkpoint_cb,
]

results = cls_model.fit(
    train_data,
    epochs=400,
    verbose=2,
    callbacks=callbacks,
    validation_data=val_data,
    steps_per_epoch=train_batches,
    validation_steps=val_batches,
    validation_freq=1,
    initial_epoch=initial_epoch)

# Evaluate

In [ ]:
# Note: you may have to reload the model (without compiling) before evaluating
# The act of compiling the model with only one loss function messes things up.

In [ ]:
val_imgs = dataset.get_image_list('validation')
len(val_imgs)

In [ ]:
y_preds = np.zeros((len(val_imgs), 4), dtype=np.float32)
y_true = np.zeros((len(val_imgs), 4), dtype=np.uint8)

for i, img_name in enumerate(val_imgs):
    img, ann = dataset.get_example_from_img_name(img_name)
    img_batch = np.expand_dims(img, axis=0)
    y_seg, y_cls = cls_model.predict(img_batch)
    
    y_true[i, :] = np.amax(ann, axis=(0, 1))
    y_preds[i, :] = y_cls[0, :]
    

In [ ]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()

In [ ]:
from sklearn.metrics import roc_curve, auc, confusion_matrix
thresholds = [0.5, 0.5, 0.5, 0.5]
for i in range(y_true.shape[-1]):
    y_preds_thresh = (y_preds > thresholds[i]).astype(np.uint8)
    cm = confusion_matrix(y_true[:, i], y_preds_thresh[:, i])
    print('Confusion matric for class {i}\n(Actual labels on left)')
    print_cm(cm, ['0', '1'])
    fpr, tpr, _ = roc_curve(y_true[:, i], y_preds[:, i])
    roc_auc = auc(fpr, tpr)
    
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange',
             lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC for class {i}')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
import cv2

def postprocess(y, y_cls, thresh=None, cls_thresh=None, min_px_area=500):
    if thresh is None:
        thresh = [0.85, 0.85, 0.85, 0.85]
    if cls_thresh is None:
        cls_thresh = [0.5, 0.5, 0.5, 0.5]

    # TODO: handle batches properly
    batches, height, width, classes = y.shape
    assert batches == 1
    
    y_argmax = np.argmax(y, axis=-1) # Only allow one class at each pixel
    y_one_hot = onehottify(y_argmax, y.shape[-1], dtype=int)
    for c in range(classes):
        y_one_hot[:, :, :, c][y[:, :, :, c] < thresh[c]] = 0 # Background
    
    for c in range(classes):
        if y_cls[0, c] < cls_thresh[c]:
            y_one_hot[:, :, :, c] = 0
        else:
            num_component, component = cv2.connectedComponents(y_one_hot[0, :, :, c].astype(np.uint8))
            for comp_idx in range(1, num_component):
                comp_mask = (component == comp_idx)
                if comp_mask.sum() < min_px_area:
                    y_one_hot[0, :, :, c][comp_mask] = 0

    return y_one_hot

def eval(model, dataset, img_list, num_classes=4, thresh=None, cls_thresh=None, verbose=False):
    num_empty_gt = [0] * num_classes
    num_empty_gt_mask_pred = [0] * num_classes
    num_mask_gt_empty_pred = [0] * num_classes
    mask_sizes = [[] for _ in range(num_classes)]

    dice_coeffs = []
    for img_name in img_list:
        img, ann = dataset.get_example_from_img_name(img_name)
        img_batch = np.expand_dims(img, axis=0)
        y, y_cls = model.predict(img_batch)
        y_one_hot = postprocess(y, y_cls, thresh=thresh, cls_thresh=cls_thresh)
        dice_coeffs.append(dice_coeff_kaggle(y_one_hot[0, :, :, :], ann))
        
        for c in range(num_classes):
            gt_mask_size = np.count_nonzero(ann[:, :, c])
            gt_is_empty = gt_mask_size == 0
            pred_is_empty = np.count_nonzero(y_one_hot[0, :, :, c]) == 0
            
            if gt_is_empty:
                num_empty_gt[c] += 1
            else:
                mask_sizes[c].append(gt_mask_size)

            if gt_is_empty and not pred_is_empty:
                num_empty_gt_mask_pred[c] += 1
            
            if not gt_is_empty and pred_is_empty:
                num_mask_gt_empty_pred[c] += 1

    if verbose:
        for c in range(num_classes):
            print(f'**** Class {c} ****')
            print(f'Num empty gt masks: {num_empty_gt[c]}')
            print(f'Num non-empty gt masks: {len(img_list) - num_empty_gt[c]}')
            print(f'Num empty gt mask and non-empty pred: {num_empty_gt_mask_pred[c]} '
                  f'({num_empty_gt_mask_pred[c] / num_empty_gt[c]})')
            print(f'Num non-empty gt mask and empty pred: {num_mask_gt_empty_pred[c]} '
                  f'({num_mask_gt_empty_pred[c] / (len(img_list) - num_empty_gt[c])})')
            print(f'Mean mask size: {np.mean(mask_sizes[c])} (stddev: {np.std(mask_sizes[c])})')
        
    mean_dice_coeff = np.mean(dice_coeffs)
    print(f'Mean dice coeff: {mean_dice_coeff}')
    return mean_dice_coeff, dice_coeffs

In [ ]:
mean_dice_coeff, dice_coeffs = eval(cls_model,
                                     dataset,
                                     val_imgs,
                                     thresh=[0.85, 0.85, 0.85, 0.85],
                                     cls_thresh=[0.5, 0.5, 0.5, 0.5])

In [ ]:
#Mean dice coeff: 0.9327411366608803
#Mean dice coeff: 0.9336561477986604

In [ ]:
indices = np.argsort(dice_coeffs)[:50] # Indices of 10 worst images
for i in indices:
    print(f'{i}: {dice_coeffs[i]}')

In [ ]:
# Visualize Image Prediction
img_id = 936
thresh = 0.85
classification_thresh = [0.5, 0.5, 0.95, 0.5]

img_name = val_imgs[img_id]
img, ann = dataset.get_example_from_img_name(img_name)
img_batch = np.expand_dims(img, axis=0)
y, y_cls = cls_model.predict(img_batch)
plt.figure(figsize=(10, 3))
plt.imshow(visualize_segmentations(np.repeat(img, 3, axis=-1), ann))
plt.show()

for i in range(4):
    plt.figure(figsize=(10, 3))
    plt.imshow(y[0, :, :, i])
    plt.show()
print(f'Mask Classification: {y_cls[0, :]}')
for i in range(y.shape[-1]):
    pred = y[0, :, :, i] > thresh
    if y_cls[0, i] < classification_thresh[i]:
        pred[:, :] = 0
    plt.figure(figsize=(10, 3))
    plt.imshow(pred)
    plt.show()

# Save HDF5 Model

In [ ]:
date_str = datetime.now().strftime("%Y%m%d-%H%M%S")
cls_model.save(f'seg_cls_model_{date_str}.h5')